<a href="https://colab.research.google.com/github/sandeeep-prajapati/pytorch_Practice_Repo/blob/main/convalusionalNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
num_epochs =5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [3]:
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform = transforms.ToTensor(),
                                           download = True)
test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train = False,
                                          transform = transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                          batch_size = batch_size,
                                          shuffle = False)

100%|██████████| 9912422/9912422 [00:00<00:00, 91647917.64it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 29473404.82it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 23426546.65it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 16352385.21it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [10]:
class ConvNet(nn.Module):
  def __init__(self,num_classes = 10):
    super(ConvNet,self).__init__()
    self.layer1 = nn.Sequential(
      nn.Conv2d(1,16,kernel_size = 5, stride =1, padding = 2),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2,stride = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(16,32, kernel_size = 5, stride = 1, padding = 2),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride= 2),
    )
    self.fc = nn.Linear(7*7*32, num_classes)
  def forward(self, x):
    out = self.layer1(x)
    out = self.layer2(out)
    out = out.reshape(out.size(0),-1)
    out = self.fc(out)
    return out

model = ConvNet(num_classes)


In [13]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [14]:
total_step = len(train_loader)

In [15]:
for epoch in range(num_epochs):
  for i,(images,labels) in enumerate(train_loader):
    outputs = model(images)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
      print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
          .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.2051
Epoch [1/5], Step [200/600], Loss: 0.1028
Epoch [1/5], Step [300/600], Loss: 0.1531
Epoch [1/5], Step [400/600], Loss: 0.0439
Epoch [1/5], Step [500/600], Loss: 0.1248
Epoch [1/5], Step [600/600], Loss: 0.0616
Epoch [2/5], Step [100/600], Loss: 0.0441
Epoch [2/5], Step [200/600], Loss: 0.0519
Epoch [2/5], Step [300/600], Loss: 0.0131
Epoch [2/5], Step [400/600], Loss: 0.0838
Epoch [2/5], Step [500/600], Loss: 0.0223
Epoch [2/5], Step [600/600], Loss: 0.0132
Epoch [3/5], Step [100/600], Loss: 0.0213
Epoch [3/5], Step [200/600], Loss: 0.0219
Epoch [3/5], Step [300/600], Loss: 0.0355
Epoch [3/5], Step [400/600], Loss: 0.0198
Epoch [3/5], Step [500/600], Loss: 0.0105
Epoch [3/5], Step [600/600], Loss: 0.0323
Epoch [4/5], Step [100/600], Loss: 0.0032
Epoch [4/5], Step [200/600], Loss: 0.0077
Epoch [4/5], Step [300/600], Loss: 0.0361
Epoch [4/5], Step [400/600], Loss: 0.0090
Epoch [4/5], Step [500/600], Loss: 0.0937
Epoch [4/5], Step [600/600], Loss:

In [16]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 98.92 %
